# ETH and Sol domains price comparaison
On part du postulat que SOL gagne fortement et va continuer de monter en marketshare.
Les domaines SOL sont underlooked and misspricé à mon avis. Ce tool va permettre de voir les ventes sur les ENS ETH et les comparer aux SNS pour obtenir les meilleures opportunités.
## Test des APIS
Il nous faut un RPC Sol pour accéder au resolver SNS.
On utilise Alchemy.

In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import os
# Define helpers to connect to Bitmex API.
import json
import urllib
import hashlib
import hmac
import urllib.parse
from datetime import timedelta, datetime
from dateutil.parser import parse
import time
import pickle
import numpy as np
load_dotenv()

True

## ENS
### Define ENS functions

In [53]:
def get_all_ens_sales(start_time: datetime, end_time: datetime):
    # query params
    params = {
        "startTimestamp": start_time.timestamp(),
        "endTimestamp": end_time.timestamp(),
        "limit": 30,
        "contract": "0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85",
        "includeTokenMetadata": True,
        'sortBy': 'price'
    }
    headers = {"x-api-key": os.getenv('RESERVOIR_API_KEY')}
    df_ens_sales_eth = pd.DataFrame()

    def loop_over_continuation(df_ens_sales_eth):
        start_time = time.time()
        resp = requests.get("https://api.reservoir.tools/sales/v6", params=params, headers=headers)
        if resp.status_code != 200:
            print('Error', resp.status_code, resp.text)
            time.sleep(3)
            return loop_over_continuation(df_ens_sales_eth)
        else:
            try:
                resp_json = resp.json()
                df_ens_sales_eth = pd.concat([df_ens_sales_eth, pd.DataFrame(resp_json['sales'])])
                sales = resp_json['sales']
                print('sale price :', sales[-1]['price']['amount']['usd'])
                print('sale timestamp :', datetime.fromtimestamp(sales[-1]['timestamp']))
                if resp_json['continuation'] and sales[-1]['price']['amount']['usd'] > 1000:
                    params['continuation'] = resp_json['continuation']
                    time.sleep(0.5-start_time if start_time<0.5 else 0)
                    return loop_over_continuation(df_ens_sales_eth)
                else:
                    print('Finished')
                    return df_ens_sales_eth
            except Exception as e:
                print(e)
                print(resp.text)
                params['continuation'] = resp_json['continuation']
                return loop_over_continuation(df_ens_sales_eth)
    return loop_over_continuation(df_ens_sales_eth)


def ens_cleaning(df):
    df_ens_sales = df.copy()
    df_ens_sales['price_eth'] = df_ens_sales.apply(lambda x: x['price']['amount']['native'] if (
                x['price']['currency']['contract'] == '0x0000000000000000000000000000000000000000' or
                x['price']['currency']['contract'] == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') else None, axis=1)
    df_ens_sales['domain'] = df_ens_sales.apply(lambda x: x['token']['name'], axis=1)
    df_ens_sales = df_ens_sales.dropna(subset=['domain'])
    df_ens_sales['domain'] = df_ens_sales.apply(lambda x: x['domain'][:-4],
                                                axis=1)
    df_ens_sales['price_usd'] = df_ens_sales.apply(lambda x: x['price']['amount']['usd'], axis=1)
    # Get average of sales for each domain
    df_ens_avg_price = df_ens_sales.groupby('domain')['price_usd'].mean().reset_index()
    df_ens_avg_price = df_ens_avg_price.rename(columns={'price_usd': 'avg_price_usd'})
    df_ens_count_sales = df_ens_sales.groupby('domain').count().reset_index()
    # Rename the count column to 'nb_sales' for clarity
    df_ens_count_sales = df_ens_count_sales.rename(columns={'price_eth': 'nb_sales'})
    df_ens_sales = df_ens_sales.merge(df_ens_count_sales[['domain', 'nb_sales']], on='domain', how='left')
    df_ens_sales = df_ens_sales.merge(df_ens_avg_price, on='domain', how='left')
    df_ens_sales.drop_duplicates(subset=['domain'], inplace=True)
    df_ens_sales = df_ens_sales[
        ['domain', 'avg_price_usd', 'nb_sales', 'timestamp', 'txHash', 'fillSource', 'orderKind']]
    return df_ens_sales

### Fetch ENS data

In [20]:
# df_ens_sales_raw=get_all_ens_sales(datetime(2021, 1, 1), datetime(2023, 12, 23))
# df_ens_sales_raw

In [21]:
# df = df_ens_sales_raw.copy()
# df['price_eth']=df.apply(lambda x: x['price']['amount']['native'] if (x['price']['currency']['contract']=='0x0000000000000000000000000000000000000000' or x['price']['currency']['contract']=='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') else None, axis=1)
# df['domain']=df.apply(lambda x:str(x['token']['name'][:-4]), axis=1)
# df['price_usd']=df.apply(lambda x: x['price']['amount']['usd'],axis=1)
# # Get average of sales for each domain
# df_ens_avg_price = df.groupby('domain')['price_usd'].mean().reset_index()
# df_ens_avg_price

### Clean ENS df and do computations

In [22]:
# ens_cleaning(df_ens_sales_raw)

## SNS
### Define SNS functions

In [2]:
# Get all SNS listings
def get_all_sns_listings():
    limit = 20
    url = "https://api-mainnet.magiceden.dev/v2/collections/bonfida/listings"
    params = {
        "limit": limit,  # Unknown limit, to be tested
        "offset": 0,
    }
    df_sns_listings = pd.DataFrame()

    def loop_over_continuation(df_sns_listings):
        start_loop = time.time()
        resp = requests.get(url, params=params)
        resp_json = resp.json()
        df_sns_listings_iteration = pd.DataFrame(resp_json)
        df_sns_listings = pd.concat([df_sns_listings, df_sns_listings_iteration])

        if not df_sns_listings_iteration.empty:
            params["offset"] += limit
            print(resp_json[-1]['price'])
            time_taken = time.time() - start_loop
            time.sleep(0.5 - time_taken if time_taken < 0.5 else 0)
            return loop_over_continuation(df_sns_listings)
        else:
            return df_sns_listings

    return loop_over_continuation(df_sns_listings)


def sns_cleaning(df):
    df_sns_listings = df.copy()
    df_sns_listings['price_sol'] = df_sns_listings['price']
    df_sns_listings['domain'] = df_sns_listings.apply(lambda x: x['token']['name'], axis=1)
    df_bitmex_hourly_sol_prices_raw = get_sol_prices()
    df_sns_listings['price_usd'] = df_sns_listings['price_sol'] * df_bitmex_hourly_sol_prices_raw.iloc[-1]['close']
    df_sns_listings = df_sns_listings[['domain', 'price_sol', 'price_usd']]
    return df_sns_listings


def check_if_registered(domain_name: str):
    # This is using QuickNode API and SNS add-on, if SNS is bought, it automatically resolves the domain to owner if bought.
    # Therefore if result = None it means domain is available.
    try:
        resp = requests.post(
            'https://late-small-voice.solana-mainnet.quiknode.pro/213109288e418132157c5d7b3da4f0fc4af3158f/', json={
                "jsonrpc": "2.0",
                "method": "sns_resolveDomain",
                "params": [
                    domain_name,
                ],
                "id": 42
            }
            )
        result = resp.json()['result']
        print('domain', domain_name, 'result', result)
        return result != None
    except Exception as e:
        print('Exception', resp.text)
        return None


def process_domain(domain):
    return domain, check_if_registered(domain)


# Get SOL historical price from mex
api_key_id = os.getenv('BITMEX_API_ID')
api_key_secret = os.getenv('BITMEX_API_SECRET')

# Simple GET
#
verb = 'GET'
# Note url-encoding on querystring - this is '/api/v1/instrument?filter={"symbol": "XBTM15"}'
# Be sure to HMAC *exactly* what is sent on the wire
path = 'https://www.bitmex.com/api/v1/instrument'
expires = 1518064237  # 2018-02-08T04:30:37Z
data = json.dumps({"filter": {"symbol": "XBTM15"}})
print(data)


def generate_signature(secret, verb, url, expires, data):
    """Generate a request signature compatible with BitMEX."""
    # Parse the url so we can remove the base and extract just the path.
    print(data)
    parsedURL = urllib.parse.urlparse(url)
    path = parsedURL.path
    if parsedURL.query:
        path = path + '?' + parsedURL.query

    if isinstance(data, (bytes, bytearray)):
        data = data.decode('utf8')

    message = bytes(verb + path + str(expires) + data, 'utf-8')

    signature = hmac.new(bytes(secret, 'utf-8'), message, digestmod=hashlib.sha256).hexdigest()
    return signature


def generate_headers(url):
    headers = {}
    sig_expires = int((datetime.now() + timedelta(minutes=1)).timestamp())
    signature = generate_signature(os.getenv('BITMEX_API_SECRET'), 'GET', url, sig_expires, '')
    headers['api-expires'] = str(sig_expires)
    headers['api-signature'] = signature
    headers['api-key'] = str(os.getenv('BITMEX_API_ID'))
    return headers


# Extract historical data for ETHUSD from Bitmex.
# Define the endpoint URL
def get_sol_prices():
    url = "https://www.bitmex.com/api/v1/trade/bucketed"
    count = 1000  # Max 1000
    cursor = 0
    # We can take full time length here because we will merge on timestamp from nft sales (which are filtered by from_time and to_time)
    start_time = "2018-03-02T09:02:00"
    end_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    # Define the parameters
    params = {
        "symbol": "SOLUSD",
        "count": count,  # Modify this value based on your needs
        "start": cursor,  # After 11000 we dont get any result.
        "binSize": "1h",
        "columns": "close,timestamp",
    }

    df_bitmex_hourly_sol_prices_raw = pd.DataFrame()

    while True:
        encoded_params = urllib.parse.urlencode(params)
        full_url = url + "?" + encoded_params
        headers = generate_headers(full_url)

        response = requests.get(full_url, headers=headers)
        print(headers)
        print(params)
        # Check the status code
        if response.status_code == 200:
            resp_json = response.json()
            # If the least recent timestamp is > of our end time we stop the loop. It means we registered all the data we need and more.
            if len(resp_json) == 0:
                print('Finished')
                break

            # Parse the response JSON into a DataFrame.
            df_bitmex_hourly_sol_prices_raw = pd.concat([df_bitmex_hourly_sol_prices_raw, pd.DataFrame(resp_json)])
            print('First row', resp_json[0])
            # Calc the remaining API calls we can make before we hit the rate limit.
            remaining_api_calls = int(response.headers.get('x-ratelimit-remaining'))
            print(remaining_api_calls)
            # Increment the cursor by count.
            cursor += count
            # For whatever reason not detailed, cursor stops at 11000. So we need to reset the start time to get the next 11000.

            params['start'] = cursor
            print('Cursor', cursor)
            # If we dont have any remaining_api_calls we wait the given time.
            if remaining_api_calls == 0:
                time_to_sleep = int(response.headers.get('x-ratelimit-reset')) - int(time.time())
                # Time can be negative, because of the time it takes to process the data if the reset limit is very close.
                if time_to_sleep > 0:
                    print(f"Request limit reached, waiting {time_to_sleep}")
                    time.sleep(time_to_sleep)
        elif response.status_code == 429:
            must_wait = int(response.headers.get('retry-after'))
            print(f"Request limit reached, waiting {must_wait}")
            time.sleep(must_wait)
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)
    return df_bitmex_hourly_sol_prices_raw

{"filter": {"symbol": "XBTM15"}}


### Execute and get SNS listings

In [24]:
# df_sns_listings_raw = get_all_sns_listings()
# df_sns_listings_raw

### Clean SNS df and do computations

In [25]:
# sns_cleaning(df_sns_listings_raw)

## Main computation
### SNS and ENS fetching concurrent

In [37]:
from concurrent.futures import ThreadPoolExecutor
# Run ETH fetching and SNS fetching concurrently
import threading

with ThreadPoolExecutor() as executor:
    future1=executor.submit(get_all_ens_sales,datetime(2021, 5, 1), datetime(2023, 12, 30))
    future2=executor.submit(get_all_sns_listings)
    df_ens_sales_raw=future1.result()  # This will wait for the thread to finish and return its result
    df_sns_listings_raw=future2.result()  # This will wait for the thread to finish and return its result

0.169
sale price : 108298.7708
sale timestamp : 2022-09-08 06:19:01
0.177
0.177
sale price : 71984.81646
sale timestamp : 2022-08-09 19:25:15
0.177
0.185
sale price : 49722.62237
sale timestamp : 2022-09-22 15:51:23
0.195
sale price : 55278.09688
sale timestamp : 2022-06-13 14:14:54
0.21
sale price : 45976.82642
sale timestamp : 2022-09-22 08:48:47
0.23
sale price : 49332.1652
sale timestamp : 2022-09-20 12:58:35
0.25
sale price : 61646.10746
sale timestamp : 2022-09-13 06:06:09
0.28
0.29
0.29
0.3
0.35
sale price : 47193.28811
sale timestamp : 2022-09-21 06:21:35
0.39111
sale price : 44182.32073
sale timestamp : 2022-09-23 12:57:35
0.39111
sale price : 43117.68649
sale timestamp : 2022-09-23 13:08:47
0.4
0.41
sale price : 41736.69948
sale timestamp : 2022-10-19 03:53:11
0.42069
sale price : 38570.50269
sale timestamp : 2022-11-14 00:22:59
0.45
0.48
sale price : 41857.16737
sale timestamp : 2022-09-17 02:59:47
0.4911
0.5
sale price : 34937.66716
sale timestamp : 2022-10-19 03:37:35
0.5


In [38]:
with open('df_ens_sales_raw.pkl', 'wb') as f:
    pickle.dump(df_ens_sales_raw, f)
with open('df_sns_listings_raw.pkl', 'wb') as f:
    pickle.dump(df_sns_listings_raw, f)


with open('df_ens_sales_raw.pkl','rb') as file:
    df_ens_sales_raw=pickle.load(file)
with open('df_sns_listings_raw.pkl','rb') as file:
    df_sns_listings_raw=pickle.load(file)

In [39]:
df_ens_sales_raw

,id,saleId,token,orderId,orderSource,orderSide,orderKind,from,to,amount,...,timestamp,price,washTradingScore,paidFullRoyalty,feeBreakdown,isDeleted,createdAt,updatedAt,marketplaceFeeBps,royaltyFeeBps
0,e8e46e67461d5dbd0dd42f9b586d96e2e91011ed097133...,7f8600d320c4cc1143d0229fce11cd7cf66cb3e7822736...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0xd75ae95ac1da33dfc1c4c3f3e0b169553456bb613212...,opensea.io,ask,wyvern-v2,0xd61415f6c5d93027778ceab5f099f64c29f58fd7,0x644bd3f3a92acea4e2e65f1d5f2b3978ce3f45fa,1,...,1633805945,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2021-10-09T18:59:05.000Z,2023-03-29T13:52:43.598Z,NaN,NaN
1,33cee605ffa67af1b61529a5ca8860235b8f0144021e34...,efd4e69b7c09091588efb88190731a6baec95dc6dd1c3b...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x3593446e12cab0649b4f7ceb06796d6732e4d09cc992...,opensea.io,ask,wyvern-v2.3,0x9394a7cf1b7ad5de4825c3a3d8033453e23a4ce1,0xb34d849c0551c6a4c0d491224bcc6114b906afc4,1,...,1656872054,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-07-03T18:14:24.177Z,2023-03-08T19:49:14.661Z,NaN,NaN
2,a0bbb73eb40a43c7acc173c49b969fc2501c5a14fd7f52...,dd9d6bd1b726bb0cfb28319e4eb346700fc7852af665e7...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0xf12ebdb647b173bee25e3dac8de31997f8cb6270223e...,opensea.io,ask,wyvern-v2.3,0xdf5e7a6dcd10b1e3ead7c2576410f9160cc691ad,0x252e00419fa7b35f4bef92b6244097acc5c220a5,1,...,1656509031,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-06-29T13:24:05.779Z,2023-03-09T08:08:35.391Z,NaN,NaN
3,91642cdab49aa78877b6f28a73f9f5db8e8a8929d664e2...,577802fea36fd8212edf25b87ce85be1891cf6e9fb70e6...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x48285a72057c3fcd62656050f64cb90a453723b4f009...,x2y2.io,ask,x2y2,0xc871015868443a6e07de4a205ccce8df4a63f5ec,0x81d6e8dfde5d9ad36a0b3c303e21cf8975661ed0,1,...,1653086759,{'currency': {'contract': '0x00000000000000000...,1,True,"[{'kind': 'marketplace', 'bps': 50, 'recipient...",False,2022-09-14T11:07:42.572Z,2023-03-10T14:52:23.408Z,50.0,NaN
4,d88b19939534995fb475dde2678b88817c9971f3183afc...,c1091dc4821340b11157f3872630dfa7e90e2627c8b341...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0xfb54fa4ead6cce787f64bcdf2a801543c809351be133...,x2y2.io,ask,x2y2,0x81d6e8dfde5d9ad36a0b3c303e21cf8975661ed0,0xc871015868443a6e07de4a205ccce8df4a63f5ec,1,...,1653087073,{'currency': {'contract': '0x00000000000000000...,1,True,"[{'kind': 'marketplace', 'bps': 50, 'recipient...",False,2022-09-14T11:07:42.572Z,2023-03-10T14:52:06.101Z,50.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,82ee254230661153b87ddd22ebacd8412049e1feb8a9b5...,10649abe4c6ca9ba58e085a0f7aba94753fc235f80a71a...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0xacf88910617b96369b8f59eb0be5bf90771121e7a738...,opensea.io,ask,wyvern-v2.3,0x92c3f55448e1476195da7aefc40a3ead18070654,0xa2af5d1d52f2cff37887e6d1ad13aefe904f7d65,1,...,1652454868,{'currency': {'contract': '0x00000000000000000...,0,True,[],False,2022-05-13T15:14:28.000Z,2023-03-11T07:23:30.418Z,NaN,NaN
26,3b580cb6d5f7aeca0806f080d6b01de7851720b60b75db...,07e0ef56a84fa0dc203f5b47a0dddac572f39426997b27...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x1092f41248d646f54934e6a84d3efce8dc75a0252503...,x2y2.io,ask,x2y2,0x409aaafb68b9ab181188ec3add2a263c596224a4,0xc5ef4d98c6fc683a738a6f95db147f3a5066b271,1,...,1656340904,{'currency': {'contract': '0x00000000000000000...,0,True,"[{'kind': 'marketplace', 'bps': 50, 'recipient...",False,2022-06-27T14:42:21.159Z,2023-03-09T12:12:41.680Z,50.0,NaN
27,22f91a3b15e6fb3c364cdd689b37e34c3245013de46e2c...,88f6722065767d8a4dd95d14b6b2b66a700490f55072ba...,{'contract': '0x57f1887a8bf19b14fc0df6fd9b2acc...,0x8d1a9a4ae58349a861fa8cbfbda04089ad90103488ed...,opensea.io,bid,seaport,0x61c4a9107de33e75ed6fe29550ae2cc29ced2264,0x24a0dac3f5a91eb33ab7b9f23f529eadd8d47fb9,1,...,1660004784,{'currency': {'contract': '0xc02aaa39b223fe8d0...,0,True,"[{'kind': 'ma

In [35]:
df_sns_listings_raw

,pdaAddress,auctionHouse,tokenAddress,tokenMint,seller,sellerReferral,tokenSize,price,priceInfo,rarity,extra,expiry,token,listingSource
0,57mPmrun2YA2TFUipdvoAFyzBdGGaCYVVYrWmhhxFVxe,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,4itKNDXizrNdJob7k2XukJk84CHY2whcyKavNu7KpySP,12iPK3JsrzNYjpFeabXiqyocNyM7p4VSexQf1rQAz6xG,G6nZjNFP5trEwD2Q5sNBYN39SZUtiSVgVvJCrmSNqf6P,autMW8SgBkVYeBgqYiTuJZnkvDZMVU2MHJh9Jh7CSQ2,1,0.137,"{'solPrice': {'rawAmount': '137000000', 'addre...",{},{'img': 'https://cloudflare-ipfs.com/ipfs/QmRw...,-1,{'mintAddress': '12iPK3JsrzNYjpFeabXiqyocNyM7p...,M2
1,GKpCcYZJ3mw5tULYbdmQLaXL9jcS3uAN5iCZr8RdWD7g,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,Gex1AHNaygmvJakJkbG73q6ECGdXaA3DfUzmeex7wGBb,7zh4oARhfikEVozabU3gWCNUKbwoM5ffmKx8qE4Mdhg7,pHY2PRWsu8CNdx4E5HvVACD5er1PcGtQqjgTBKVX9xz,AjhorLY77TDVgbC3JMXsr6gKvXwf7A1eqsms58gKttK4,1,0.137,"{'solPrice': {'rawAmount': '137000000', 'addre...","{'moonrank': {'rank': 3717, 'absolute_rarity':...",{'img': 'https://cloudflare-ipfs.com/ipfs/QmSX...,-1,{'mintAddress': '7zh4oARhfikEVozabU3gWCNUKbwoM...,M2
2,9esm4ufzjMieEXC46H37AkNVev3sgNNYMEszi9YZUKGN,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,GDHUWdDu63zdjGgqQ4znEMardP8QEcsz3YebqDxr2paa,5qwCV1DvWsZzdmgH684o9TKmM6Xjh77RseoR8LqjktwH,HMXAwrizHwq8JR8rVTjzo6dfw8EDUexVnvoEXkZs1XT3,autMW8SgBkVYeBgqYiTuJZnkvDZMVU2MHJh9Jh7CSQ2,1,0.141,"{'solPrice': {'rawAmount': '141000000', 'addre...",{},{'img': 'https://cloudflare-ipfs.com/ipfs/QmVu...,-1,{'mintAddress': '5qwCV1DvWsZzdmgH684o9TKmM6Xjh...,M2
3,BD9XfzDfqGDMHYmjY9zFSRAvsaUgRYuy7v8ipBdEF8ae,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,5YPDGuByY95QxDGFhcH91iFozaEsmqBbWzCwcsbHAC1J,2qA3zwPZft1HPpfN7bFQp6DHduP57PxiiKHRye6HEoxA,2v3inkUm9ksx6kkAHnVj33LYyYK8UbiKcyiCd8XXNuh8,autMW8SgBkVYeBgqYiTuJZnkvDZMVU2MHJh9Jh7CSQ2,1,0.153,"{'solPrice': {'rawAmount': '153000000', 'addre...",{},{'img': 'https://cloudflare-ipfs.com/ipfs/Qmcf...,-1,{'mintAddress': '2qA3zwPZft1HPpfN7bFQp6DHduP57...,M2
4,FHiYXMeTriZh9eq8DepVoaBsFrMuE4YTCEJ7jBRJiU3g,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,GqyWWeCjpV3KWS5pjWp1TQ1T6sTbNjLkUde4Rnm8SMEn,786iCTLgEoBtWGWTurcrs7QdiBEGM3QAeRXcgRiXNsNE,HMXAwrizHwq8JR8rVTjzo6dfw8EDUexVnvoEXkZs1XT3,autMW8SgBkVYeBgqYiTuJZnkvDZMVU2MHJh9Jh7CSQ2,1,0.153,"{'solPrice': {'rawAmount': '153000000', 'addre...",{},{'img': 'https://cloudflare-ipfs.com/ipfs/QmdN...,-1,{'mintAddress': '786iCTLgEoBtWGWTurcrs7QdiBEGM...,M2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,59Peh7UvCrEsBiBmBGxayUv4T7F1f18N2VUsDN3kkYrH,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,3rhEnFWor8PYnr6MbbwqQhmvfS5zeAipir2GpFA8bmbG,9gxsYCQKeQkVASZgcYTbjDm3sGxswxrfmSb5AD1hdjmU,5vNJfkEdqFaVjsBiUAhEQmxWNg57nWiqscq4P8JQtFyK,autMW8SgBkVYeBgqYiTuJZnkvDZMVU2MHJh9Jh7CSQ2,1,8000000.000,"{'solPrice': {'rawAmount': '8000000000000000',...",{},{'img': 'https://cloudflare-ipfs.com/ipfs/QmR5...,-1,{'mintAddress': '9gxsYCQKeQkVASZgcYTbjDm3sGxsw...,M2
0,BfHGJ2h63y3VVHbL392d4JdqDgEdBHJRUkzEbjWP9Wjd,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,3htpb53ck4sYAZnQtcwihwUbTbc4oeDcGGnh2DacRyL6,CjL6vpxzwqAA7ovEghyWS7SVo7YNUQXbmZxMGynXMFFG,7oJZjDKatxm9YTbjPYgRZfT5f1eQnNqqR4RsbxJAXC45,autMW8SgBkVYeBgqYiTuJZnkvDZMVU2MHJh9Jh7CSQ2,1,8000000.000,"{'solPrice': {'rawAmount': '8000000000000000',...","{'moonrank': {'rank': 3361, 'absolute_rarity':...",{'img': 'https://cloudflare-ipfs.com/ipfs/Qmct...,-1,{'mintAddress': 'CjL6vpxzwqAA7ovEghyWS7SVo7YNU...,M2
1,6mgjLGqHchjrWTjsJbmXdZYALWXRShAJZLLMoKeBdNuA,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,EKTRW8wWSexXspvj7BytMysbsjy3hQoxMJEPaG2CU53W,DLhPeodsrSDDy89yTUhA7AVomx68q7n2xEUdLz4VpN2s,EirVV6qsdoRf5VojSs547XW5hLwexJc9fK247xQ6maMm,autMW8SgBkVYeBgqYiTuJZnkvDZMVU2MHJh9Jh7CSQ2,1,8000000.000,"{'solPrice': {'rawAmount': '8000000000000000',...","{'moonrank': {'rank': 9756, 'absolute_rarity':...",{'img': 'https://cloudflare-ipfs.com/ipfs/QmNZ...,-1,{'mintAddress': 'DLhPeodsrSDDy89yTUhA7AVomx68q...,M2
2,5Nog56gv97hjvN7Hd4WczqFkXdz1GHrQ336JWmPoGEtx,E8cU1WiRWjanGxmn96ewBgk9vPTcL6AEZ1t6F6fkgUWe,HsKxHq29xZPQtRSU8KDJGKRprD

### SNS and ENS df cleaning, computation and merging

In [105]:
df_sns_listings = sns_cleaning(df_sns_listings_raw)
df_ens_sales = ens_cleaning(df_ens_sales_raw)

df_sales = df_ens_sales.merge(df_sns_listings, how='left', on='domain')
df_sales


{'api-expires': '1704221239', 'api-signature': 'f8a37bc610fa3943489318a5e868d3d45e0c73215a76bb60d1b2b3206cd06d6d', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 0, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'timestamp': '2021-10-20T04:00:00.000Z', 'symbol': 'SOLUSD'}
119
Cursor 1000

{'api-expires': '1704221239', 'api-signature': 'a77a7aee49276d0e089f8ca1fc127e8f0fa1e26daa18a489e35b67dc6b1b46f6', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 1000, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'symbol': 'SOLUSD', 'timestamp': '2021-11-30T20:00:00.000Z', 'close': 215.45}
118
Cursor 2000

{'api-expires': '1704221240', 'api-signature': 'a444515756157907c39df9cc83ab33d70d98f6acd1bc88b8186e867e61e20739', 'api-key': 'HM_OVkhTCygs5BcVk2NY6aIi'}
{'symbol': 'SOLUSD', 'count': 1000, 'start': 2000, 'binSize': '1h', 'columns': 'close,timestamp'}
First row {'symbol': 'SOLUSD', 'timestamp': '2022-0

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd
0,paradigm,1.494591e+06,1,1633805945,0xb55725377362589127683a5279cc71f5fa7db13efb79...,opensea.io,wyvern-v2,NaN,NaN
1,000,3.205625e+05,1,1656872054,0x659f6961fb48db842f061250f6ac28e858e11b211d46...,opensea.io,wyvern-v2.3,2000.0,215640.0
2,porno,2.104102e+05,1,1656509031,0x64e33539108a54f905d0a082575bdc197a34bbdd59a1...,opensea.io,wyvern-v2.3,NaN,NaN
3,03jul,2.609897e+05,7,1653086759,0x8f0bf710aee447fb089187a4774c3cba4743f5ed032b...,x2y2.io,x2y2,NaN,NaN
4,sapdeutschland,1.799394e+05,1,1673820875,0xd71af68474266518c216f7405623648d7eb6604e1cf1...,opensea.io,seaport,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7239,2398,1.522683e+03,1,1676657951,0x3df8ac45b4dbfaf5d315adb56439aebcf7c964efdf1b...,opensea.io,seaport,NaN,NaN
7240,6317,2.606178e+03,1,1651351319,0x8996fb206c6b13fe6e3f89b4e13372349cff5d302323...,gem.xyz,wyvern-v2.3,NaN,NaN
7241,whey,3.038153e+03,1,1648504643,0x4e1d0d406980eb0572a2a9738f12641bd3e0e84731c4...,opensea.io,wyvern-v2.3,NaN,NaN
7242,4451,1.819197e+03,1,1652454868,0xeda6ee9f679f3c5060d734f7a6573ee451025482b034...,opensea.io,wyvern-v2.3,NaN,NaN


### Get registrability of domains on Bonfida
We only try to get registrability for ENS domains sold above 1k.

In [86]:
# All ENS sales matched on its unlisted counterpart on MagicEden (could be unlisted or available in the registry)
df_unlisted_sns_domains = df_sales[df_sales['price_sol'].isna()]
df_unlisted_sns_domains

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd,is_registered,delta
9,❤‍🔥.et,155249.64890,1,1658646924,0xa047eef884d9973c6403bc5753e3068ce5ef8fcb5454...,opensea.io,seaport,NaN,20.0,False,155229.64890
10,٠٠٠.et,115334.83150,1,1656988610,0xc11ffec811565a963d452ea899f9f4a65a41e2c338e4...,opensea.io,seaport,NaN,20.0,False,115314.83150
33,٠٠٧.et,59103.40249,1,1656911595,0x5fed3ab5997766758763f36ebc0accdf954699b581ea...,opensea.io,wyvern-v2.3,NaN,20.0,False,59083.40249
43,٠٠٩.et,57655.88227,1,1656991445,0x38d8b4eddb62b255e4dd1b5339845c8a60bf3187ca08...,opensea.io,seaport,NaN,20.0,False,57635.88227
102,c6opoil,56089.97541,1,1674811655,0x4b7d80a842876046dab447a9139cfac38456351cc688...,looksrare.org,looks-rare,NaN,20.0,False,56069.97541
...,...,...,...,...,...,...,...,...,...,...,...
7238,1775,1450.51343,1,1678056743,0x71e2a089dc4402baadb537db2e4309b15c1ef052acf8...,ens.vision,seaport,NaN,NaN,True,NaN
7239,2398,1522.68274,1,1676657951,0x3df8ac45b4dbfaf5d315adb56439aebcf7c964efdf1b...,opensea.io,seaport,NaN,NaN,True,NaN
7240,6317,2606.17814,1,1651351319,0x8996fb206c6b13fe6e3f89b4e13372349cff5d302323...,gem.xyz,wyvern-v2.3,NaN,NaN,True,NaN
7241,whey,3038.15265,1,1648504643,0x4e1d0d406980eb0572a2a9738f12641bd3e0e84731c4...,opensea.io,wyvern-v2.3,NaN,NaN,True,NaN


In [87]:
# Check unregistered SNS for ENS in between 10k and 1k
df_filtered_sales = df_unlisted_sns_domains[(df_unlisted_sns_domains['avg_price_usd']>1000)]

In [3]:
# Check registration for SNS
def check_registrability_sns(domains:list):

    # Create a ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=2) as executor:
        # Submit tasks to the executor
        futures = [executor.submit(process_domain, domain) for domain in domains]

        # Create an empty dictionary to store results
        domain_status = {}

        # Retrieve results as they are completed
        for future in as_completed(futures):
            domain, is_registered = future.result()
            domain_status[domain] = is_registered
        return domain_status

In [148]:
domains = df_filtered_sales['domain'].tolist()
domains

domainException {"jsonrpc":"2.0","result":null,"id":42}
domain baruc result None
domain 955 result 3dMmM2WaNS9ktiGC5tG4DGcDPQ1eDAPtzj7dLFnUyXKh
domain triventures result None
domain ٣٢٨.et result None


['❤\u200d🔥.et',
 '٠٠٠.et',
 '٠٠٧.et',
 '٠٠٩.et',
 'c6opoil',
 '⌐◨-◨.et',
 '57810',
 '24⁄7',
 'serendibite',
 '🚀🚀🚀.et',
 '🤵\u200d♂.et',
 '🤡🤡🤡.et',
 '7⃣7.et',
 '£££.et',
 '🖕🏾🖕🏾.et',
 '👩\u200d🦲.et',
 '😎😎😎.et',
 '0x8200',
 'curvefi',
 'ᴇᴛʜᴇʀᴇᴜᴍ.et',
 '🤦\u200d♂.et',
 '0️⃣2️⃣.et',
 'cmla',
 '🙇\u200d♂.et',
 '🙅\u200d♂.et',
 '👩\u200d🔧.et',
 '🧑\u200d🦱.et',
 'eth111',
 '💆\u200d♂.et',
 '435453',
 '👩\u200d👩\u200d👦\u200d👦.et',
 '💇\u200d♂.et',
 '👨\u200d👦.et',
 '🤹\u200d♀.et',
 '3️⃣5️⃣.et',
 '🧎\u200d♂.et',
 '💲💲💲.et',
 'fht86',
 '2️⃣1️⃣',
 '🐻\u200d❄',
 '🧖\u200d♀.et',
 '0️⃣1️⃣.et',
 '😚😚😚.et',
 '3️⃣8️⃣.et',
 '👿👿👿.et',
 '٠١٠.et',
 '٥٥٥.et',
 '٣٣٣.et',
 '🧛🧛🧛.et',
 'mainli',
 '🔑🔑🔑.et',
 'easili',
 '•5746.et',
 'bitcry',
 '377797',
 '👍👍👍.et',
 '🤑🤑🤑.et',
 'awafn',
 '٤٤٤.et',
 'ethercapsules',
 'azukielemental',
 '🏊\u200d♀.et',
 '•5776.et',
 'sproto-pepes',
 '55-88-88',
 'area61',
 '🚀🚀🚀🚀.et',
 'smurfsociety',
 '🧎\u200d♀.et',
 'wavyhair',
 'neopunks',
 '1ethis1eth',
 'trusttheforce',
 '1ethequal1eth',
 '🧜\u200d♂

In [144]:
# Convert the results dictionary back to a DataFrame
df_sns_to_be_registered = pd.DataFrame(list(check_registrability_sns(domains).items()), columns=['domain', 'is_registered'])

domainException {"jsonrpc":"2.0","result":null,"id":42}
domain 👨‍❤‍💋‍👨🏼.et result None
domain benandjerrys result None
domain 六七五.et result None
domain 🙋🏿‍♀.et result None
domain ٧٨١.et result None
domain siamsquare result None
domain ١٣٣.et result None
domain liono result None
domain dondo result None
domain 🤩🤩🤩🤩.et result None
domain internships result None
domain 七四零.et result None
domain ٥٠٨.et result None
domain baibu result None
domainException {"jsonrpc":"2.0","result":null,"id":42}
domain kissmyface result None
domain ٨٣٤.et result None
domain ٨١٦.et result None
domain 🤹‍♂.et result None
domain accepts result None


KeyboardInterrupt: 

In [83]:
# Filter not listed domains
df_sns_to_be_registered

,domain,is_registered
0,paradigm,True
1,03jul,True
2,porno,True
3,ensmaxisnft,True
4,sapdeutschland,True
...,...,...
6859,2398,True
6860,7396,True
6861,6317,True
6862,whey,True


In [106]:
# Merge sns registered and sales
df_sales=df_sales.merge(df_sns_to_be_registered, how='left', on='domain')
# df_sns_to_be_registered contains only the unlisted domains, so we fill NaN value with True as the other domains are listed therefore registered.
df_sales['is_registered'].fillna(True, inplace=True)
df_sales['is_registered']=df_sales.apply(lambda x: x['is_registered']==True, axis=1)

# There is an error with our parsing of domain, it keeps .et in it, remove those rows
df_sales = df_sales[~df_sales['domain'].str.endswith('.et')]

# Convert timestamp to readable format
df_sales['timestamp']=df_sales.apply(lambda x: datetime.fromtimestamp(x['timestamp']), axis=1)

# We suppose that unregistered domains cost 20$ (which is not rly true for rare domains etc..)
df_sales['price_usd']=df_sales.apply(lambda x:x['price_usd'] if x['is_registered'] else 20, axis=1)
df_sales

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd,is_registered
0,paradigm,1.494591e+06,1,2021-10-09 21:59:05,0xb55725377362589127683a5279cc71f5fa7db13efb79...,opensea.io,wyvern-v2,NaN,NaN,True
1,000,3.205625e+05,1,2022-07-03 21:14:14,0x659f6961fb48db842f061250f6ac28e858e11b211d46...,opensea.io,wyvern-v2.3,2000.0,215640.0,True
2,porno,2.104102e+05,1,2022-06-29 16:23:51,0x64e33539108a54f905d0a082575bdc197a34bbdd59a1...,opensea.io,wyvern-v2.3,NaN,NaN,True
3,03jul,2.609897e+05,7,2022-05-21 01:45:59,0x8f0bf710aee447fb089187a4774c3cba4743f5ed032b...,x2y2.io,x2y2,NaN,NaN,True
4,sapdeutschland,1.799394e+05,1,2023-01-16 00:14:35,0xd71af68474266518c216f7405623648d7eb6604e1cf1...,opensea.io,seaport,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...
7239,2398,1.522683e+03,1,2023-02-17 20:19:11,0x3df8ac45b4dbfaf5d315adb56439aebcf7c964efdf1b...,opensea.io,seaport,NaN,NaN,True
7240,6317,2.606178e+03,1,2022-04-30 23:41:59,0x8996fb206c6b13fe6e3f89b4e13372349cff5d302323...,gem.xyz,wyvern-v2.3,NaN,NaN,True
7241,whey,3.038153e+03,1,2022-03-29 00:57:23,0x4e1d0d406980eb0572a2a9738f12641bd3e0e84731c4...,opensea.io,wyvern-v2.3,NaN,NaN,True
7242,4451,1.819197e+03,1,2022-05-13 18:14:28,0xeda6ee9f679f3c5060d734f7a6573ee451025482b034...,opensea.io,wyvern-v2.3,NaN,NaN,True


## Data management
### Dump in Pickle

In [107]:
with open('df_sales.pkl', 'wb') as f:
    pickle.dump(df_sales, f)

### Load from disk

In [108]:
with open('df_sales.pkl','rb') as file:
    df_sales=pickle.load(file)

In [109]:
df_sales

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd,is_registered
0,paradigm,1.494591e+06,1,2021-10-09 21:59:05,0xb55725377362589127683a5279cc71f5fa7db13efb79...,opensea.io,wyvern-v2,NaN,NaN,True
1,000,3.205625e+05,1,2022-07-03 21:14:14,0x659f6961fb48db842f061250f6ac28e858e11b211d46...,opensea.io,wyvern-v2.3,2000.0,215640.0,True
2,porno,2.104102e+05,1,2022-06-29 16:23:51,0x64e33539108a54f905d0a082575bdc197a34bbdd59a1...,opensea.io,wyvern-v2.3,NaN,NaN,True
3,03jul,2.609897e+05,7,2022-05-21 01:45:59,0x8f0bf710aee447fb089187a4774c3cba4743f5ed032b...,x2y2.io,x2y2,NaN,NaN,True
4,sapdeutschland,1.799394e+05,1,2023-01-16 00:14:35,0xd71af68474266518c216f7405623648d7eb6604e1cf1...,opensea.io,seaport,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...
7239,2398,1.522683e+03,1,2023-02-17 20:19:11,0x3df8ac45b4dbfaf5d315adb56439aebcf7c964efdf1b...,opensea.io,seaport,NaN,NaN,True
7240,6317,2.606178e+03,1,2022-04-30 23:41:59,0x8996fb206c6b13fe6e3f89b4e13372349cff5d302323...,gem.xyz,wyvern-v2.3,NaN,NaN,True
7241,whey,3.038153e+03,1,2022-03-29 00:57:23,0x4e1d0d406980eb0572a2a9738f12641bd3e0e84731c4...,opensea.io,wyvern-v2.3,NaN,NaN,True
7242,4451,1.819197e+03,1,2022-05-13 18:14:28,0xeda6ee9f679f3c5060d734f7a6573ee451025482b034...,opensea.io,wyvern-v2.3,NaN,NaN,True


## Data analysis
### Compute delta of SNS listed and avg sale price of ENS

In [110]:
# Filter available domains with the max delta between ENS and SNS
df_sales['delta'] = df_sales.apply(lambda x: x['avg_price_usd']-x['price_usd'],axis=1)
df_sales.sort_values(by=['delta','price_usd'], ascending=[False,True], inplace=True)
df_sales

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd,is_registered,delta
1,000,320562.450900,1,2022-07-03 21:14:14,0x659f6961fb48db842f061250f6ac28e858e11b211d46...,opensea.io,wyvern-v2.3,2000.0,215640.00,True,104922.450900
102,c6opoil,56089.975410,1,2023-01-27 11:27:35,0x4b7d80a842876046dab447a9139cfac38456351cc688...,looksrare.org,looks-rare,NaN,20.00,False,56069.975410
32,010,71585.463400,1,2022-11-11 21:57:59,0xcca57b1decf340424230c9ba1ca96fd444943fa9de08...,x2y2.io,x2y2,149.0,16065.18,True,55520.283400
266,57810,55488.512304,34,2022-05-07 09:59:14,0xe0c5eb7fe43ac7d47ac63fb3c933c3c973a09a2a9bae...,x2y2.io,x2y2,NaN,20.00,False,55468.512304
252,24⁄7,48860.234190,1,2022-05-22 08:22:50,0x817ba88a454e0c45aad288909a3ef3768b8c711f1e43...,opensea.io,wyvern-v2.3,NaN,20.00,False,48840.234190
...,...,...,...,...,...,...,...,...,...,...,...
7239,2398,1522.682740,1,2023-02-17 20:19:11,0x3df8ac45b4dbfaf5d315adb56439aebcf7c964efdf1b...,opensea.io,seaport,NaN,NaN,True,NaN
7240,6317,2606.178140,1,2022-04-30 23:41:59,0x8996fb206c6b13fe6e3f89b4e13372349cff5d302323...,gem.xyz,wyvern-v2.3,NaN,NaN,True,NaN
7241,whey,3038.152650,1,2022-03-29 00:57:23,0x4e1d0d406980eb0572a2a9738f12641bd3e0e84731c4...,opensea.io,wyvern-v2.3,NaN,NaN,True,NaN
7242,4451,1819.196730,1,2022-05-13 18:14:28,0xeda6ee9f679f3c5060d734f7a6573ee451025482b034...,opensea.io,wyvern-v2.3,NaN,NaN,True,NaN


### Compute delta of SNS unregistered and avg sale price of ENS, CAREFUL SNS registration price varies from the assumed 20$

In [111]:
# Get only unregistered domains
df_sales_unregistered = df_sales[df_sales['is_registered']==False]
df_sales_unregistered

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd,is_registered,delta
102,c6opoil,56089.975410,1,2023-01-27 11:27:35,0x4b7d80a842876046dab447a9139cfac38456351cc688...,looksrare.org,looks-rare,NaN,20.0,False,56069.975410
266,57810,55488.512304,34,2022-05-07 09:59:14,0xe0c5eb7fe43ac7d47ac63fb3c933c3c973a09a2a9bae...,x2y2.io,x2y2,NaN,20.0,False,55468.512304
252,24⁄7,48860.234190,1,2022-05-22 08:22:50,0x817ba88a454e0c45aad288909a3ef3768b8c711f1e43...,opensea.io,wyvern-v2.3,NaN,20.0,False,48840.234190
222,serendibite,43910.826630,0,2023-02-22 20:26:11,0x7b14b6d82ef50d806d329e81ec658f24edc87f2d5b19...,None,zeroex-v2,NaN,20.0,False,43890.826630
395,0x8200,30702.140210,1,2023-05-11 19:09:23,0x03b34da116a2693be47da9220336c77d5929420dfba5...,opensea.io,seaport-v1.5,NaN,20.0,False,30682.140210
...,...,...,...,...,...,...,...,...,...,...,...
6910,0xvitalik,1111.855890,1,2022-06-21 00:28:14,0x682cbae95a3b1c5f2e71ee741e7417f0fcd51febe2b4...,opensea.io,wyvern-v2.3,NaN,20.0,False,1091.855890
7187,h0v,1091.440360,1,2022-07-12 00:46:12,0xb4711b5596c8725fb82446741806d3c1984873458794...,ens.domains,mint,NaN,20.0,False,1071.440360
6757,2xj,1077.191350,1,2022-07-04 09:34:29,0xd6ca4c982c36de54bc753d31aaba1380184881ee1f7e...,ens.domains,mint,NaN,20.0,False,1057.191350
7016,c7d,1030.940000,1,2022-06-17 12:09:56,0x8f8b65162a4e58b3e7e92ab2abe79fb982bae1cafbd1...,ens.domains,mint,NaN,20.0,False,1010.940000


In [115]:
# Get only listed SNS < 2000$
df_sales_listed = df_sales[(df_sales['is_registered']==True) & (df_sales['price_usd']<2000)]
df_sales_listed

,domain,avg_price_usd,nb_sales,timestamp,txHash,fillSource,orderKind,price_sol,price_usd,is_registered,delta
253,ashokchakra,26923.185190,1,2022-06-30 22:05:21,0x3fc76c1e7ee08030fb9781920c2e28a2bdbf2f7f2e7a...,opensea.io,seaport,1.0,107.82,True,26815.365190
481,guccifit,25182.796840,0,2023-05-21 02:13:23,0x8ceedbd879d00177e93bf45eaef8304d8c2979039dd3...,opensea.io,seaport-v1.5,0.5,53.91,True,25128.886840
450,strongmind,25230.170610,1,2022-08-02 01:12:15,0xd84280eb76b10e59015a84789b489ec13a796da2d818...,opensea.io,seaport,2.0,215.64,True,25014.530610
408,friendtechno,25308.190930,1,2023-09-26 05:48:35,0x7b03a24a00980cdff624759b869150c544a82b3ad2e3...,x2y2.io,x2y2,10.0,1078.20,True,24229.990930
528,deepsense,20130.605690,1,2023-09-19 13:59:35,0x67201fbeeeeb220fa83e99bf003cf77cee152d3c1319...,x2y2.io,x2y2,0.5,53.91,True,20076.695690
...,...,...,...,...,...,...,...,...,...,...,...
5349,9823,1309.241410,2,2022-07-01 17:22:04,0xb2fbe68b4ef9b0496e1cde233ab24f7e0727c6905a78...,opensea.io,seaport,13.0,1401.66,True,-92.418590
5754,4335,1584.048620,1,2022-12-16 16:20:47,0x3d1a6b7c0ed3454f6d742756cbdfadc0ac794a569dae...,opensea.io,seaport,16.0,1725.12,True,-141.071380
6655,vacancies,1676.830480,1,2022-09-09 20:55:51,0x59e2bf3181239eb9c7126817833194911f1ecd23b926...,opensea.io,seaport,17.0,1832.94,True,-156.109520
5228,2583,1391.259455,2,2022-07-15 20:41:48,0x52eefacdc228ce65027da9d8d606ebee8b39e45ce00b...,opensea.io,seaport,15.0,1617.30,True,-226.040545


# Check non registered english words

In [4]:
with open('english_words.json') as f:
    english_words = json.load(f)

In [5]:
domains_english = english_words['commonWords']
domains_english

['a',
 'able',
 'about',
 'absolute',
 'accept',
 'account',
 'achieve',
 'across',
 'act',
 'active',
 'actual',
 'add',
 'address',
 'admit',
 'advertise',
 'affect',
 'afford',
 'after',
 'afternoon',
 'again',
 'against',
 'age',
 'agent',
 'ago',
 'agree',
 'air',
 'all',
 'allow',
 'almost',
 'along',
 'already',
 'alright',
 'also',
 'although',
 'always',
 'america',
 'amount',
 'and',
 'another',
 'answer',
 'any',
 'apart',
 'apparent',
 'appear',
 'apply',
 'appoint',
 'approach',
 'appropriate',
 'area',
 'argue',
 'arm',
 'around',
 'arrange',
 'art',
 'as',
 'ask',
 'associate',
 'assume',
 'at',
 'attend',
 'authority',
 'available',
 'aware',
 'away',
 'awful',
 'baby',
 'back',
 'bad',
 'bag',
 'balance',
 'ball',
 'bank',
 'bar',
 'base',
 'basis',
 'be',
 'bear',
 'beat',
 'beauty',
 'because',
 'become',
 'bed',
 'before',
 'begin',
 'behind',
 'believe',
 'benefit',
 'best',
 'bet',
 'between',
 'big',
 'bill',
 'birth',
 'bit',
 'black',
 'bloke',
 'blood',
 'blow

In [6]:
df_sns_unregistered_english_words = pd.DataFrame(list(check_registrability_sns(domains_english).items()), columns=['domain', 'is_registered'])

domain able result 5VnR4AzPJw9RfCWbmsaFvNfQuzu7Srz3HFyUvKiPqnx8
domain a result DY4v61XYV7Tmf9YVWkxvFLUn3V3rccukz6Jewq7CgGCN
domain about result 8CRGA6XMCsitY93bEPG7zNyYaB5fZVQfauneadVyCRro
domain absolute result 8pphYmdknXpMtQmLFSJYYvpK9bc2S5C5VZE3QDKakF9c
domain account result CuMM9LLBbpvSp1sNx6SFZrRopb5XASKsJVsJdPYYH4ge
domain accept result 46szE6TBQrdCySCUPJHR6xzNCaVge6WnUhkpwnajDXep
domain across result 2Gn592tnoXtkndEQ8N8Yudip3TLD24172JZm3yEeJKRu
domain achieve result BuzNHwNweCgcHyJDYG9tvwAXhnGi6ySTtRfoTBsNUveH
domain act result Fyrvi7SMTy3QECP17hbDdpQsErzfA6vJGgSUxQjUiaiK
domain active result 323gPPQBJG8oeaY5mj4tmLQeJXmJtr5ZPWWPoY4HAP18
domain actual result 8s2mNhTFmijHfv8eRSwghSZMxkvwJhmPDK2dJtAQRgzL
domain add result 2uLh1v1MuoVcGVVR73eMx9X8hNQzPKFrMNGK3yJGyzVi
domain address result CuMM9LLBbpvSp1sNx6SFZrRopb5XASKsJVsJdPYYH4ge
domain admit result 6gka5LEuJz9DvMBTDsFRvgbRmtfUJEmMo2vrxjPBNTUK
domain advertise result B9KiiJoiJoJGG66DArz3kTYwCpJWiiS9egHsrpZLK5LR
domain affect res

In [7]:
df_sns_unregistered_english_words[(df_sns_unregistered_english_words['is_registered']==False) | (df_sns_unregistered_english_words['is_registered'].isna())]

,domain,is_registered
42,apparent,False
149,Christ,False
150,Christmas,False
160,closes,False
180,confer,False
276,especial,False
305,favour,False
565,occasion,False
587,otherwise,False
636,practise,False


In [8]:
with open('french_names.json','r') as f:
    french_names = json.load(f)['names']

In [9]:
french_names=[name.lower() for name in french_names]

In [10]:
df_sns_unregistered_french_names = pd.DataFrame(list(check_registrability_sns(french_names).items()), columns=['domain', 'is_registered'])

domain adèle result None
domain alex result HuB5nXrX3AY1eCw688hXGdDgNcfWmZL74kXs7VUxLZFw
domain alice result 94uQzuwDPacwttxcor7TnHfAgS5qNv5kv6MXPwJgZBpq
domain anna result 4zEVxBQkAdTpuK3Kc2AkmZGJAHCay6s4qNL3gpEjhpSg
domain apolline result 9REKBMHy7uMu1kJH55HCnv2bPnd7wSQzC9LtnXJ6f3H7
domain augustin result HRbo6zYcYkA9Ne1xjBLz4dfLBWViQe68jSGFJfw7AnHW
domain candice result 8HJWU8nyHtF5KrvUp8BevcbQek5rJvpR6eTcrbiAUNtk
domain arnaud result EpRzHtRutvc6QYjJWP7zfy51ZW4Ehwk937cFkbbseSV3
domain aaron result 9MEBjAxH7qf5Gbmrd7oaJ2LWS5CvgQRwbgRXEpvthGZ3
domain alicia result BpdHg6hmpkuUicr8SUbTdzGSKoxC95XrBfKxkan5dEbh
domain alix result 91bVs3GfJ9gnpwJyxTerNn8DE5EubuwVX2XXKgcgknmi
domain andrea result GUDz4weQA2AbdLt1T3vYXyg61wV7tAUrTpM1Tsoo8JSj
domain antonin result 2gnMPjFJU5oaFWHqx9U8Lsf6AMWgX9ppvB9uzPq7V6ck
domain audrey result DsJw9WFCmWtpXmjx5i9nbQ1dH6rg8BfPecyfP9FaTNaY
domain diego result 7vWSqSw1eCXZXXUubuHWssXELNQ8MLaDgAs2ErEfCKxn
domain éloïse result None
domain emilien result B9pQ4k

In [12]:
df_sns_unregistered_french_names[df_sns_unregistered_french_names['is_registered']==False]

,domain,is_registered
0,adèle,False
15,éloïse,False
18,clémentine,False
24,angèle,False
29,amandine,False
35,amélie,False
37,aurélien,False
38,capucine,False
47,gabin,False
49,flavie,False
